# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

spark = SparkSession.builder.appName("Data wrangling with Spark SQL").getOrCreate()

user_log = spark.read.json("data/sparkify_log_small.json")

user_log.createOrReplaceTempView("user_log")


# Question 1

Which page did user id ""(empty string) NOT visit?

In [5]:
spark.sql("""
    SELECT DISTINCT page 
    FROM user_log 
    WHERE page NOT IN (
        SELECT DISTINCT page 
        FROM user_log 
        WHERE userId = ''
    )
""").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

## Response

We should use SQL over data frames because of:
 - It is straight forward. You just specify in plain SQL what data you need.
 - SQL is a common language when talking about databases, it's more likely you don't have a much greater learning curve
 
We should use data frames over SQL because of:
 - When you need to use more advanced features like adding a column to a dataframe, windowing results

# Question 3

How many female users do we have in the data set?

In [6]:
spark.sql("""
    SELECT gender, COUNT(DISTINCT userId) AS count
    FROM user_log 
    WHERE gender = 'F'
    GROUP BY gender
""").show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [11]:
spark.sql("""
    SELECT artist, COUNT(ts) AS plays_count
    FROM user_log
    WHERE page = 'NextSong'
    GROUP BY artist
    ORDER BY plays_count DESC
    LIMIT 1
""").show()

+--------+-----------+
|  artist|plays_count|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [18]:
is_home = spark.sql("""
    SELECT 
        userID, 
        page, 
        ts, 
        (CASE WHEN page = 'Home' THEN 1 ELSE 0 END) AS is_home 
    FROM 
        user_log
    WHERE 
        (page = 'NextSong') or (page = 'Home')
""")

is_home.createOrReplaceTempView("is_home_table")

cumulative_sum = spark.sql("""
    SELECT 
        *, 
        SUM(is_home) OVER (
            PARTITION BY userID 
            ORDER BY ts DESC 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS period
    FROM is_home_table
""")

cumulative_sum.createOrReplaceTempView("period_table")

spark.sql("""
    SELECT 
        AVG(count_results) 
    FROM (
        SELECT 
            COUNT(*) AS count_results 
        FROM 
            period_table 
        GROUP BY 
            userID, period, page 
        HAVING 
            page = 'NextSong'
    ) AS counts
""").show()


+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

